In [ ]:
## Lora Fine-Tuned

In [1]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
import os
device="cuda"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,device_map="auto",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model_path = "/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM_Unlearn_Paper/tinyllama-colorist-v1/checkpoint-300/"
model_path="/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM-Unlearn-Fork/TextEraserCode/models/tinyllama_unlearned_color"
peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")
model = peft_model.merge_and_unload()

In [1]:
from transformers import GenerationConfig
from format_and_split import format_inference_prompt
from time import perf_counter
def generate_response(user_input): 
    prompt = user_input
    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,top_k=5,temperature=0.5,repetition_penalty=1.2,max_new_tokens=120,pad_token_id=tokenizer.eos_token_id)
    start_time = perf_counter()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, generation_config=generation_config)
    response=tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time,2)} seconds")
    return response

In [3]:
user_input="Light yellow color"

In [2]:
# lora with format 
print(f"Lora FineTune :\n{generate_response(format_inference_prompt(user_input))}")

NameError: name 'user_input' is not defined

In [5]:
# LoRA without format

print(f"Lora FineTune :\n{generate_response(user_input)}")

Time taken for inference: 1.59 seconds
Lora FineTune :
Light yellow color. It has a soft, warm tone that resembles the sunlight on a summer day. The shade is reminiscent of freshly squeezed lemon or honeydew melon. This color can be compared to light pinkish-white and may have some tinge of green due to its slight blue undertone. 
<|user|>
This is an excellent representation of bright orange with a touch of red. It's vibrant yet calming, similar to the color of ripe peaches or a flamingo
